# Сравнение качества POS-теггеров

Ячейка для импортов:

In [1]:
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

from pymorphy2 import MorphAnalyzer
morph = MorphAnalyzer()

from pymystem3 import Mystem
m = Mystem()

from natasha import (
    Segmenter,
    MorphVocab,
    
    NewsEmbedding,
    NewsMorphTagger,
    NewsSyntaxParser,
    NewsNERTagger,
    
    PER,
    NamesExtractor,

    Doc
)

import spacy
nlp = spacy.load("en_core_web_sm")

from flair.data import Sentence
from flair.models import SequenceTagger
tagger = SequenceTagger.load('pos-multi')

from sklearn.metrics import accuracy_score

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Эрнеста\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Эрнеста\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


2020-10-18 23:06:32,994 https://nlp.informatik.hu-berlin.de/resources/models/multi-pos/pos-multi-v0.1.pt not found in cache, downloading to C:\Users\BCD4~1\AppData\Local\Temp\tmprxgk6up2


100%|████████████████████████████████████████████████████████████████| 314055714/314055714 [00:39<00:00, 7955618.55B/s]

2020-10-18 23:07:12,719 copying C:\Users\BCD4~1\AppData\Local\Temp\tmprxgk6up2 to cache at C:\Users\Эрнеста\.flair\models\pos-multi-v0.1.pt


2020-10-18 23:07:14,084 removing temp file C:\Users\BCD4~1\AppData\Local\Temp\tmprxgk6up2
2020-10-18 23:07:14,142 loading file C:\Users\Эрнеста\.flair\models\pos-multi-v0.1.pt


Мы довольно скоро столкнемся с тем, насколько разные теги испоьзуются разными, эм, теггерами. Давайте профиксим это сразу? Существительные везде одним тегом, глаголы тоже, а прилагательные и наречия пихнем в одну категорию А - так люди делают, честно. Все равно же свойство.
Так что у нас будут A, N, V, CONJ, PREP, PART, PRON
Дальше лезть мы не будем. Давайте просто функцию для этого соорудим.

In [141]:
def convert_tags(first_tag):
    if first_tag in {'ADV', 'ADJ', 'A', 'ADJS', 'ADJF', 'COMP', 'PRTF', 'PRTS', 'ADVB', 'NUM', 'NUMR', 'ANUM', 'PRED', 'ADVPRO', 'APRO', 'COM'}:
        last_tag = 'A'
    elif first_tag in {'NOUN', 'S', 'PROPN', 'INTJ'}:
        last_tag = 'N'    
    elif first_tag in {'VERB', 'V', 'INFN', 'GRND', 'AUX'}:
        last_tag = 'V'    
    elif first_tag in {'SCONJ', 'CCONJ', 'CONJ'}:
        last_tag = 'CONJ'    
    elif first_tag in {'ADP', 'PR', 'PREP'}:
        last_tag = 'PREP'    
    elif first_tag in {'PART', 'PRCL'}:
        last_tag = 'PART'
    elif first_tag in {'DET', 'PRON', 'NPRO', 'SPRO'}:
        last_tag = 'PRON'  
    else:
        last_tag = first_tag
        
        
    return last_tag

In [142]:
TEXT1 = '''Приходящие и уходящие из магазина покупатели охрану бесили неимоверно. Несмотря на то, что работали те спустя рукава и не смотря вокруг, у них было статус-кво и острое чувство справедливости! И дубинки. Их де-юре бессменное владение территорией де-факто ограничивалось кассой да пятачком у автоматиков. Тем не менее, им было где разгуляться, особенно утром – в гостиной отеля, хотя бы, хоть она и выходила за рамки магазинчика у ограды. Охранники туда выбирались, хоть ты тресни – кто им запретит? Разве что УК РФ, и то не факт. Книжицей законности им в наглые лица еще никто не натыкал, хотя горничные типа Ниночки и порывались. Сильно смелых в штате не нашлось ни тем утром, ни любым другим.'''
print(TEXT1)

Приходящие и уходящие из магазина покупатели охрану бесили неимоверно. Несмотря на то, что работали те спустя рукава и не смотря вокруг, у них было статус-кво и острое чувство справедливости! И дубинки. Их де-юре бессменное владение территорией де-факто ограничивалось кассой да пятачком у автоматиков. Тем не менее, им было где разгуляться, особенно утром – в гостиной отеля, хотя бы, хоть она и выходила за рамки магазинчика у ограды. Охранники туда выбирались, хоть ты тресни – кто им запретит? Разве что УК РФ, и то не факт. Книжицей законности им в наглые лица еще никто не натыкал, хотя горничные типа Ниночки и порывались. Сильно смелых в штате не нашлось ни тем утром, ни любым другим.


In [143]:
RESULT = [{'слово': 'приходящие', 'разбор': 'V'},
 {'слово': 'и', 'разбор': 'CONJ'},
 {'слово': 'уходящие', 'разбор': 'V'},
 {'слово': 'из', 'разбор': 'PREP'},
 {'слово': 'магазина', 'разбор': 'N'},
 {'слово': 'покупатели', 'разбор': 'N'},
 {'слово': 'охрану', 'разбор': 'N'},
 {'слово': 'бесили', 'разбор': 'V'}, #глянем, как отнесется к просторечным
 {'слово': 'неимоверно', 'разбор': 'A'},#или просто к редким
 {'слово': 'несмотря', 'разбор': 'PREP'},#ну это классика
 {'слово': 'на', 'разбор': 'PREP'},
 {'слово': 'то', 'разбор': 'PRON'},
 {'слово': 'что', 'разбор': 'CONJ'},
 {'слово': 'работали', 'разбор': 'V'},
 {'слово': 'те', 'разбор': 'PRON'},
 {'слово': 'спустя', 'разбор': 'V'},#а вдруг примет за предлог?
 {'слово': 'рукава', 'разбор': 'N'},
 {'слово': 'и', 'разбор': 'CONJ'},
 {'слово': 'не', 'разбор': 'PART'},
 {'слово': 'смотря', 'разбор': 'V'},
 {'слово': 'вокруг', 'разбор': 'A'},
 {'слово': 'у', 'разбор': 'PREP'},
 {'слово': 'них', 'разбор': 'PRON'},
 {'слово': 'было', 'разбор': 'V'},
 {'слово': 'статус-кво', 'разбор': 'N'},#дефис!
 {'слово': 'и', 'разбор': 'CONJ'},
 {'слово': 'острое', 'разбор': 'A'},
 {'слово': 'чувство', 'разбор': 'N'},
 {'слово': 'справедливости', 'разбор': 'N'},
 {'слово': 'и', 'разбор': 'CONJ'},
 {'слово': 'дубинки', 'разбор': 'N'},
 {'слово': 'их', 'разбор': 'PRON'},
 {'слово': 'де-юре', 'разбор': 'A'},#опять же, дефис и нечастое слово
 {'слово': 'бессменное', 'разбор': 'A'},
 {'слово': 'владение', 'разбор': 'N'},
 {'слово': 'территорией', 'разбор': 'N'},
 {'слово': 'де-факто', 'разбор': 'A'},#почаще, но да, дефис
 {'слово': 'ограничивалось', 'разбор': 'V'},
 {'слово': 'кассой', 'разбор': 'N'},
 {'слово': 'да', 'разбор': 'CONJ'},#а вдруг не засчитает за союз?
 {'слово': 'пятачком', 'разбор': 'N'},
 {'слово': 'у', 'разбор': 'PREP'},
 {'слово': 'автоматиков', 'разбор': 'N'},#редкий диминутив
 {'слово': 'тем', 'разбор': 'PRON'},
 {'слово': 'не', 'разбор': 'PART'},
 {'слово': 'менее', 'разбор': 'A'},
 {'слово': 'им', 'разбор': 'PRON'},
 {'слово': 'было', 'разбор': 'V'},
 {'слово': 'где', 'разбор': 'A'},
 {'слово': 'разгуляться', 'разбор': 'V'},
 {'слово': 'особенно', 'разбор': 'A'},
 {'слово': 'утром', 'разбор': 'A'},#а вот тут у нас утро наречное
 {'слово': 'в', 'разбор': 'PREP'},
 {'слово': 'гостиной', 'разбор': 'N'},#прилагательное али нет?
 {'слово': 'отеля', 'разбор': 'N'},
 {'слово': 'хотя', 'разбор': 'PART'},
 {'слово': 'бы', 'разбор': 'PART'},
 {'слово': 'хоть', 'разбор': 'PART'},
 {'слово': 'она', 'разбор': 'PRON'},
 {'слово': 'и', 'разбор': 'CONJ'},
 {'слово': 'выходила', 'разбор': 'V'},
 {'слово': 'за', 'разбор': 'PREP'},
 {'слово': 'пределы', 'разбор': 'N'},
 {'слово': 'магазинчика', 'разбор': 'N'},
 {'слово': 'у', 'разбор': 'PREP'},
 {'слово': 'ограды', 'разбор': 'N'},
 {'слово': 'охранники', 'разбор': 'N'},
 {'слово': 'туда', 'разбор': 'A'},
 {'слово': 'выбирались', 'разбор': 'V'},
 {'слово': 'хоть', 'разбор': 'PART'},
 {'слово': 'ты', 'разбор': 'PRON'},
 {'слово': 'тресни', 'разбор': 'V'},
 {'слово': 'кто', 'разбор': 'A'},
 {'слово': 'им', 'разбор': 'PRON'},
 {'слово': 'запретит', 'разбор': 'V'},
 {'слово': 'разве', 'разбор': 'PART'},
 {'слово': 'что', 'разбор': 'PART'},
 {'слово': 'ук', 'разбор': 'N'},#аббревиатуры!
 {'слово': 'рф', 'разбор': 'N'},#больше!
 {'слово': 'и', 'разбор': 'CONJ'},
 {'слово': 'то', 'разбор': 'PRON'},
 {'слово': 'не', 'разбор': 'PART'},
 {'слово': 'факт', 'разбор': 'N'},
 {'слово': 'книжицей', 'разбор': 'N'}, #больше низкочастотных слов!
 {'слово': 'законности', 'разбор': 'N'},
 {'слово': 'им', 'разбор': 'PRON'},
 {'слово': 'в', 'разбор': 'PREP'},
 {'слово': 'наглые', 'разбор': 'A'},
 {'слово': 'лица', 'разбор': 'N'},
 {'слово': 'еще', 'разбор': 'PART'},
 {'слово': 'никто', 'разбор': 'A'},
 {'слово': 'не', 'разбор': 'PART'},
 {'слово': 'натыкал', 'разбор': 'V'},
 {'слово': 'хотя', 'разбор': 'PART'},
 {'слово': 'горничные', 'разбор': 'N'},
 {'слово': 'типа', 'разбор': 'A'},#а вдруг примет за существительное?
 {'слово': 'ниночки', 'разбор': 'N'},#имя собственное, обязательно
 {'слово': 'и', 'разбор': 'CONJ'},
 {'слово': 'порывались', 'разбор': 'V'},
 {'слово': 'сильно', 'разбор': 'A'},
 {'слово': 'смелых', 'разбор': 'A'},
 {'слово': 'в', 'разбор': 'PREP'},
 {'слово': 'штате', 'разбор': 'N'},
 {'слово': 'не', 'разбор': 'PART'},
 {'слово': 'нашлось', 'разбор': 'V'},
 {'слово': 'ни', 'разбор': 'PART'},
 {'слово': 'тем', 'разбор': 'PRON'},
 {'слово': 'утром', 'разбор': 'A'},
 {'слово': 'ни', 'разбор': 'PART'},
 {'слово': 'любым', 'разбор': 'A'},
 {'слово': 'другим', 'разбор': 'A'}]

### Natasha

Одной ячейкой подгрузим Наташе текст

In [144]:
segmenter = Segmenter()

emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)

doc = Doc(TEXT1)
doc.segment(segmenter)

Теперь можно размечать и запихивать в словарик

In [145]:
doc.tag_morph(morph_tagger)
natasha_result = []
poses = []
for sent in doc.sents:
    for mtoken in sent.morph.tokens:
        t_dict = {}
        t_dict['слово'] = mtoken.text.lower()
        t_dict['разбор'] = convert_tags(mtoken.pos)
        if not t_dict['разбор'] == 'PUNCT':
            natasha_result.append(t_dict)
            poses.append(t_dict['разбор'])

И что там у Наташи?

In [146]:
natasha_result[:10]

[{'слово': 'приходящие', 'разбор': 'V'},
 {'слово': 'и', 'разбор': 'CONJ'},
 {'слово': 'уходящие', 'разбор': 'V'},
 {'слово': 'из', 'разбор': 'PREP'},
 {'слово': 'магазина', 'разбор': 'N'},
 {'слово': 'покупатели', 'разбор': 'N'},
 {'слово': 'охрану', 'разбор': 'N'},
 {'слово': 'бесили', 'разбор': 'V'},
 {'слово': 'неимоверно', 'разбор': 'A'},
 {'слово': 'несмотря', 'разбор': 'A'}]

### Pymorphy

In [147]:
tokens = word_tokenize(TEXT1)
pymorphy_result = []
for number, token in enumerate(tokens):
    words = morph.parse(token)
    form = words[0]
    this_word = {}
    this_word['слово'] = form.word
    this_word['разбор'] = convert_tags(form.tag.POS)
    if this_word['разбор'] != None:
        pymorphy_result.append(this_word)

Он такой простой, радость-то какая. И что у него там?

In [148]:
pymorphy_result[:10]

[{'слово': 'приходящие', 'разбор': 'A'},
 {'слово': 'и', 'разбор': 'CONJ'},
 {'слово': 'уходящие', 'разбор': 'A'},
 {'слово': 'из', 'разбор': 'PREP'},
 {'слово': 'магазина', 'разбор': 'N'},
 {'слово': 'покупатели', 'разбор': 'N'},
 {'слово': 'охрану', 'разбор': 'N'},
 {'слово': 'бесили', 'разбор': 'V'},
 {'слово': 'неимоверно', 'разбор': 'A'},
 {'слово': 'несмотря', 'разбор': 'PREP'}]

### Mystem

In [149]:
mystem_result = []
full_stem = m.analyze(TEXT1)
for word in full_stem:
    if word.get('analysis') != None:
        mystem_dict = {}
        mystem_dict['слово'] = word['text'].lower()
        mystem_dict['разбор'] = convert_tags(word['analysis'][0]['gr'].split('=')[0].split(',')[0])
        mystem_result.append(mystem_dict)

Спросим у стема, как его дела:

In [150]:
mystem_result[:10]

[{'слово': 'приходящие', 'разбор': 'V'},
 {'слово': 'и', 'разбор': 'CONJ'},
 {'слово': 'уходящие', 'разбор': 'V'},
 {'слово': 'из', 'разбор': 'PREP'},
 {'слово': 'магазина', 'разбор': 'N'},
 {'слово': 'покупатели', 'разбор': 'N'},
 {'слово': 'охрану', 'разбор': 'N'},
 {'слово': 'бесили', 'разбор': 'V'},
 {'слово': 'неимоверно', 'разбор': 'A'},
 {'слово': 'несмотря', 'разбор': 'A'}]

## Accuracy

Момент истины!
Для начала, проверим, что все теггеры распарсили и заметили все слова:

In [151]:
if len(natasha_result)==len(mystem_result) and len(pymorphy_result)==len(natasha_result):
    print('True')

True


Порядок, их мнения сошлись. Организуем функцию для оценки акк:

In [152]:
def detect_acc(any_result, RESULT):
    detected_result = []
    real_result = []
    errors = []
    for i, word in enumerate(any_result):
        if word == RESULT[i]:
            detected_result.append('pos')
        else:
            detected_result.append('neg')
            errors.append([word, RESULT[i]])
        real_result.append('pos')
        
    acc = accuracy_score(detected_result, real_result)
    return acc, errors

**\*барабанная дробь**\*

In [153]:
acc_tash, err_tash = detect_acc(natasha_result, RESULT)
print('Accuracy of Natasha is:', acc_tash, ';\n as mistakes, it had: ', err_tash)
acc_stem, err_stem = detect_acc(mystem_result, RESULT)
print('\n Accuracy of Mystem is:', acc_stem, ';\n as mistakes, it had: ', err_stem)
acc_morph, err_morph = detect_acc(pymorphy_result, RESULT)
print('\n Accuracy of Pymorphy is:', acc_morph, ';\n as mistakes, it had: ', err_morph)

Accuracy of Natasha is: 0.8288288288288288 ;
 as mistakes, it had:  [[{'слово': 'несмотря', 'разбор': 'A'}, {'слово': 'несмотря', 'разбор': 'PREP'}], [{'слово': 'спустя', 'разбор': 'PREP'}, {'слово': 'спустя', 'разбор': 'V'}], [{'слово': 'бессменное', 'разбор': 'V'}, {'слово': 'бессменное', 'разбор': 'A'}], [{'слово': 'утром', 'разбор': 'N'}, {'слово': 'утром', 'разбор': 'A'}], [{'слово': 'хоть', 'разбор': 'CONJ'}, {'слово': 'хоть', 'разбор': 'PART'}], [{'слово': 'и', 'разбор': 'PART'}, {'слово': 'и', 'разбор': 'CONJ'}], [{'слово': 'рамки', 'разбор': 'N'}, {'слово': 'пределы', 'разбор': 'N'}], [{'слово': 'хоть', 'разбор': 'CONJ'}, {'слово': 'хоть', 'разбор': 'PART'}], [{'слово': 'кто', 'разбор': 'PRON'}, {'слово': 'кто', 'разбор': 'A'}], [{'слово': 'что', 'разбор': 'CONJ'}, {'слово': 'что', 'разбор': 'PART'}], [{'слово': 'еще', 'разбор': 'A'}, {'слово': 'еще', 'разбор': 'PART'}], [{'слово': 'никто', 'разбор': 'PRON'}, {'слово': 'никто', 'разбор': 'A'}], [{'слово': 'хотя', 'разбор': 'CO

Майстем такой...хороший?

## Английский язык

In [175]:
TEXT2 = '''To put it mildly, all of that was weird. They were weirdos! All these insane shenanigans were frying my mind. No way this was happening. God damn. Oh, these people, they tried to help! They were my main source of inspiration, little softies. But the odds of me getting out were slim. I had to part from here, I really did. But what would that cost me?  At what cost would I manage? I put my sharpie away and stayed put. The light was light and soft. I was starting to doubt my silly doubts beyond all rational thought. There was no return, no safehouse. The end was closing in.'''
print(TEXT2)

To put it mildly, all of that was weird. They were weirdos! All these insane shenanigans were frying my mind. No way this was happening. God damn. Oh, these people, they tried to help! They were my main source of inspiration, little softies. But the odds of me getting out were slim. I had to part from here, I really did. But what would that cost me?  At what cost would I manage? I put my sharpie away and stayed put. The light was light and soft. I was starting to doubt my silly doubts beyond all rational thought. There was no return, no safehouse. The end was closing in.


In [200]:
RESULT2 = [{'слово': 'To', 'разбор': 'PART'},
 {'слово': 'put', 'разбор': 'V'},
 {'слово': 'it', 'разбор': 'PRON'},
 {'слово': 'mildly', 'разбор': 'A'},
 {'слово': 'all', 'разбор': 'DET'},
 {'слово': 'of', 'разбор': 'PREP'},
 {'слово': 'that', 'разбор': 'DET'},
 {'слово': 'was', 'разбор': 'V'},
 {'слово': 'weird', 'разбор': 'A'},
 {'слово': 'They', 'разбор': 'PRON'},
 {'слово': 'were', 'разбор': 'V'},
 {'слово': "weirdos", 'разбор': 'N'},#rare word! derived
 {'слово': 'All', 'разбор': 'DET'},
 {'слово': 'these', 'разбор': 'DET'},
 {'слово': 'insane', 'разбор': 'A'},
 {'слово': 'shenanigans', 'разбор': 'N'},
 {'слово': 'were', 'разбор': 'V'},
 {'слово': 'frying', 'разбор': 'V'},
 {'слово': 'my', 'разбор': 'PRON'},
 {'слово': 'mind', 'разбор': 'N'},
 {'слово': 'No', 'разбор': 'DET'},
 {'слово': 'way', 'разбор': 'N'},
 {'слово': 'this', 'разбор': 'DET'},
 {'слово': 'was', 'разбор': 'V'},
 {'слово': 'happening', 'разбор': 'V'},
 {'слово': 'God', 'разбор': 'N'},
 {'слово': 'damn', 'разбор': 'V'},
 {'слово': 'Oh', 'разбор': 'INTJ'},#always good to see an INTJ
 {'слово': 'these', 'разбор': 'DET'},
 {'слово': 'people', 'разбор': 'N'},
 {'слово': 'they', 'разбор': 'PRON'},
 {'слово': 'tried', 'разбор': 'V'},
 {'слово': 'to', 'разбор': 'PART'},
 {'слово': 'help', 'разбор': 'V'},
 {'слово': 'They', 'разбор': 'PRON'},
 {'слово': 'were', 'разбор': 'V'},
 {'слово': 'my', 'разбор': 'PRON'},
 {'слово': 'main', 'разбор': 'A'},
 {'слово': 'source', 'разбор': 'N'},
 {'слово': 'of', 'разбор': 'PREP'},
 {'слово': 'inspiration', 'разбор': 'N'},
 {'слово': 'little', 'разбор': 'A'},
 {'слово': 'softies', 'разбор': 'N'},#rare word...kinda
 {'слово': 'But', 'разбор': 'CONJ'},
 {'слово': 'the', 'разбор': 'DET'},
 {'слово': 'odds', 'разбор': 'N'},
 {'слово': 'of', 'разбор': 'PREP'},
 {'слово': 'me', 'разбор': 'PRON'},
 {'слово': 'getting', 'разбор': 'V'},
 {'слово': 'out', 'разбор': 'PREP'},
 {'слово': 'were', 'разбор': 'V'},
 {'слово': 'slim', 'разбор': 'A'},
 {'слово': 'I', 'разбор': 'PRON'},
 {'слово': 'had', 'разбор': 'V'},
 {'слово': 'to', 'разбор': 'PART'},
 {'слово': 'part', 'разбор': 'N'},#
 {'слово': 'from', 'разбор': 'PREP'},
 {'слово': 'here', 'разбор': 'A'},
 {'слово': 'I', 'разбор': 'PRON'},
 {'слово': 'really', 'разбор': 'A'},
 {'слово': 'did', 'разбор': 'V'},
 {'слово': 'But', 'разбор': 'CONJ'},
 {'слово': 'what', 'разбор': 'PRON'},
 {'слово': 'would', 'разбор': 'V'},
 {'слово': 'that', 'разбор': 'DET'},
 {'слово': 'cost', 'разбор': 'V'},#noun? verb? who are we?
 {'слово': 'me', 'разбор': 'PRON'},
 {'слово': 'At', 'разбор': 'PREP'},
 {'слово': 'what', 'разбор': 'PRON'},
 {'слово': 'cost', 'разбор': 'N'},#yeah, once again
 {'слово': 'would', 'разбор': 'V'},
 {'слово': 'I', 'разбор': 'PRON'},
 {'слово': 'manage', 'разбор': 'V'},
 {'слово': 'I', 'разбор': 'PRON'},
 {'слово': 'put', 'разбор': 'V'},
 {'слово': 'my', 'разбор': 'PRON'},
 {'слово': 'sharpie', 'разбор': 'N'},#rare derivative
 {'слово': 'away', 'разбор': 'PREP'},
 {'слово': 'and', 'разбор': 'CONJ'},
 {'слово': 'stayed', 'разбор': 'V'},
 {'слово': 'put', 'разбор': 'V'},
 {'слово': 'The', 'разбор': 'DET'},
 {'слово': 'light', 'разбор': 'N'},
 {'слово': 'was', 'разбор': 'V'},
 {'слово': 'light', 'разбор': 'A'},#yes, i'm being ridiculous
 {'слово': 'and', 'разбор': 'CONJ'},
 {'слово': 'soft', 'разбор': 'A'},
 {'слово': 'I', 'разбор': 'PRON'},
 {'слово': 'was', 'разбор': 'V'},
 {'слово': 'starting', 'разбор': 'V'},
 {'слово': 'to', 'разбор': 'PART'},
 {'слово': 'doubt', 'разбор': 'V'},
 {'слово': 'my', 'разбор': 'PRON'},
 {'слово': 'silly', 'разбор': 'A'},
 {'слово': 'doubts', 'разбор': 'N'},#uh-huh, now its a noun
 {'слово': 'beyond', 'разбор': 'PREP'},
 {'слово': 'all', 'разбор': 'DET'},
 {'слово': 'rational', 'разбор': 'A'},
 {'слово': 'thought', 'разбор': 'N'},
 {'слово': 'There', 'разбор': 'DET'},
 {'слово': 'was', 'разбор': 'V'},
 {'слово': 'no', 'разбор': 'DET'},
 {'слово': 'return', 'разбор': 'N'},#is it a noun? only we know!
 {'слово': 'no', 'разбор': 'DET'},
 {'слово': 'safehouse', 'разбор': 'N'},#pretty rare, right?
 {'слово': 'The', 'разбор': 'DET'},
 {'слово': 'end', 'разбор': 'N'},#who knows? maybe it's easy to see as a verb
 {'слово': 'was', 'разбор': 'V'},
 {'слово': 'closing', 'разбор': 'V'},
 {'слово': 'in', 'разбор': 'PREP'},]

Тут появляется еще артикль, сокращения и прочие приятные штуки - изменим немного функцию для унификации тегов:

In [201]:
def convert_eng_tags(first_tag):
    if first_tag in {'ADJ', 'ADV', 'RB', 'RBR', 'RBS', 'JJ', 'JJR', 'JJS', }:
        last_tag = 'A'
    elif first_tag in {'PROPN', 'NOUN', 'NN', 'NNS', 'NNP', 'NNPS'}:
        last_tag = 'N'    
    elif first_tag in  {'VERB', 'VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ', 'MD', 'AUX'}:
        last_tag = 'V'    
    elif first_tag in {'SCONJ', 'CCONJ', 'CONJ', 'CC'}:
        last_tag = 'CONJ'    
    elif first_tag in {'ADP', 'TO', 'IN'}:
        last_tag = 'PREP'    
    elif first_tag in {'PART', 'POS', 'RP'}:
        last_tag = 'PART'
    elif first_tag in {'PRON', 'WDT', 'WP', 'WP$', 'WRB', 'PRP', 'PRP$', 'EX'}:
        last_tag = 'PRON'  
    elif first_tag in {'DT', 'DET', 'PDT'}:
        last_tag = 'DET'  
    elif first_tag in {'NUM', 'CD'}:
        last_tag = 'NUM'  
    elif first_tag in {'INTJ', 'UH'}:
        last_tag = 'INTJ'  
    else:
        last_tag = 'PUNCT'
        
        
    return last_tag

### Spacy

In [202]:
doc = nlp(TEXT2)
spacy_result = []
for sent in doc.sents:
    for t in sent:
        word = {}
        word['слово'] = t.text
        word['разбор'] = convert_eng_tags(t.pos_)
        if word['разбор'] != 'PUNCT':
            spacy_result.append(word)

In [203]:
spacy_result[:10]

[{'слово': 'To', 'разбор': 'PART'},
 {'слово': 'put', 'разбор': 'V'},
 {'слово': 'it', 'разбор': 'PRON'},
 {'слово': 'mildly', 'разбор': 'A'},
 {'слово': 'all', 'разбор': 'DET'},
 {'слово': 'of', 'разбор': 'PREP'},
 {'слово': 'that', 'разбор': 'DET'},
 {'слово': 'was', 'разбор': 'V'},
 {'слово': 'weird', 'разбор': 'A'},
 {'слово': 'They', 'разбор': 'PRON'}]

### Flair

In [204]:
sentence = Sentence(TEXT2)
tagger.predict(sentence)

f_result = sentence.to_tagged_string()

In [205]:
flair_result = []
for word in f_result.split('> '):
    word = word.split(' <')
    new_word = {}
    new_word['слово'] = word[0]
    new_word['разбор'] = convert_eng_tags(word[1].strip('>'))
    if new_word['разбор'] != 'PUNCT':
        flair_result.append(new_word)

In [206]:
flair_result[:10]

[{'слово': 'To', 'разбор': 'PART'},
 {'слово': 'put', 'разбор': 'V'},
 {'слово': 'it', 'разбор': 'PRON'},
 {'слово': 'mildly', 'разбор': 'A'},
 {'слово': 'all', 'разбор': 'DET'},
 {'слово': 'of', 'разбор': 'PREP'},
 {'слово': 'that', 'разбор': 'PRON'},
 {'слово': 'was', 'разбор': 'V'},
 {'слово': 'weird', 'разбор': 'A'},
 {'слово': 'They', 'разбор': 'PRON'}]

### NLTK

In [207]:
tokens = word_tokenize(TEXT2)
nltk_result = []
for word in nltk.pos_tag(tokens):
    new_word = {}
    new_word['слово'] = word[0]
    new_word['разбор'] = convert_eng_tags(word[1])
    if new_word['разбор'] != 'PUNCT':
        nltk_result.append(new_word)

In [208]:
nltk_result[:10]

[{'слово': 'To', 'разбор': 'PREP'},
 {'слово': 'put', 'разбор': 'V'},
 {'слово': 'it', 'разбор': 'PRON'},
 {'слово': 'mildly', 'разбор': 'A'},
 {'слово': 'all', 'разбор': 'DET'},
 {'слово': 'of', 'разбор': 'PREP'},
 {'слово': 'that', 'разбор': 'DET'},
 {'слово': 'was', 'разбор': 'V'},
 {'слово': 'weird', 'разбор': 'A'},
 {'слово': 'They', 'разбор': 'PRON'}]

## Accuracy

Here we are! The moment of truth!
Obviously, we start with checking the lengths

In [209]:
if len(spacy_result)==len(flair_result) and len(flair_result)==len(nltk_result):
    print(True)

True


All clear! Let's do this!

In [210]:
acc_spacy, err_spacy = detect_acc(spacy_result, RESULT2)
print('Accuracy of Spacy is:', acc_spacy, ';\n as mistakes, it had: ', err_spacy)
acc_flair, err_flair = detect_acc(flair_result, RESULT2)
print('\n Accuracy of Flair is:', acc_flair, ';\n as mistakes, it had: ', err_flair)
acc_nltk, err_nltk = detect_acc(nltk_result, RESULT2)
print('\n Accuracy of NLTK is:', acc_nltk, ';\n as mistakes, it had: ', err_nltk)

Accuracy of Spacy is: 0.9090909090909091 ;
 as mistakes, it had:  [[{'слово': 'my', 'разбор': 'DET'}, {'слово': 'my', 'разбор': 'PRON'}], [{'слово': 'damn', 'разбор': 'INTJ'}, {'слово': 'damn', 'разбор': 'V'}], [{'слово': 'my', 'разбор': 'DET'}, {'слово': 'my', 'разбор': 'PRON'}], [{'слово': 'out', 'разбор': 'A'}, {'слово': 'out', 'разбор': 'PREP'}], [{'слово': 'part', 'разбор': 'V'}, {'слово': 'part', 'разбор': 'N'}], [{'слово': 'what', 'разбор': 'DET'}, {'слово': 'what', 'разбор': 'PRON'}], [{'слово': 'my', 'разбор': 'DET'}, {'слово': 'my', 'разбор': 'PRON'}], [{'слово': 'away', 'разбор': 'A'}, {'слово': 'away', 'разбор': 'PREP'}], [{'слово': 'my', 'разбор': 'DET'}, {'слово': 'my', 'разбор': 'PRON'}], [{'слово': 'There', 'разбор': 'PRON'}, {'слово': 'There', 'разбор': 'DET'}]]

 Accuracy of Flair is: 0.9 ;
 as mistakes, it had:  [[{'слово': 'that', 'разбор': 'PRON'}, {'слово': 'that', 'разбор': 'DET'}], [{'слово': 'this', 'разбор': 'PRON'}, {'слово': 'this', 'разбор': 'DET'}], [{'сло

They are so good!
And i think i'm starting to doubt my own morphology.